# Création de rasters

**writers.gdal** de PDAL permet de créer un fichier raster à partir d'un nuage de points.
Pour rappel, un raster est une grille composée de cellules organisées en lignes et en colonnes (comme un tableau ou une image).

https://pdal.io/en/latest/stages/writers.gdal.html

## Raster élévation

Le raster est souvent utilisé pour transcire l'altitude sous la forme d'un DEM (digital elevation model - MNT modèle numérique de terrain en français) permettant ensuite de produire un ombrage ou des courbes de niveau.

In [2]:
%%time
import pathlib, json, pdal

#chemin du fichier source
datasource = pathlib.Path.cwd() /'data'/'input'
output = pathlib.Path.cwd() /'data'/'output'/'raster'
file = '2560_1145.las'

#pipeline pdal
elevation = {
 "pipeline":[
     
    #lecture du fichier source
     {
        "type" : "readers.las",
        "filename" : str(datasource/file)
    },
     
    #filtre pour ne garder que les points de la classe 2 ground
    {
        "limits" : "classification[2:2]",
        "type" : "filters.range",
    },
     
    #production d'un raster selon l'intensité
    {
        "filename":str(output)+"\\"+"elevation.tif",
        "gdaldriver":"GTiff",
        "output_type":"mean",
        "resolution":"1",
        "type": "writers.gdal"
    }
]   
}

pipeline = pdal.Pipeline(json.dumps(elevation))
pipeline.execute()

CPU times: total: 9.56 s
Wall time: 9.58 s


7240396

le résultat produit :

![elevation](data/img/pdal_writergdal_elevation.png)

Après un traitement sur Qgis pour remplir les cellules vides et produire un ombrage.

![ombrage](data/img/pdal_writergdal_elevation_ombrage.png)

## Raster intensité

Nous pouvons choisir quelle dimension du nuage nous voulons transcrire dans le raster avec l'attribut **dimension**.
L'idée est d'utiliser l'intensité des points (au lieu de z, l'élévation par défaut) pour produire une image visuelle. L'intensité varie selon la matière rencontrée par le laser et permet ainsi de produire une image en noir et blanc.

In [4]:
#pipeline pdal
intensity = {
 "pipeline":[
     
    #lecture du fichier source
     {
        "type" : "readers.las",
        "filename" : str(datasource/file)
    },
     
    #production d'un raster selon l'intensité
    {
        "filename":str(output)+"\\"+"intensity.tif",
        "gdaldriver":"GTiff",
        "dimension":"intensity",
        "output_type":"mean",
        "resolution":"1",
        "type": "writers.gdal"
    }
]   
}

pipeline = pdal.Pipeline(json.dumps(intensity))
pipeline.execute()

29525816

le résultat produit :

![intensity](data/img/pdal_writergdal_intensity.png)

Nous distinguons la forêt, les prés, les routes, les habiations etc. 